In [2]:
import pandas as pd

df = pd.read_csv("data/FakeNewsNet.csv")
print(df.head())


                                               title  \
0  Kandi Burruss Explodes Over Rape Accusation on...   
1  People's Choice Awards 2018: The best red carp...   
2  Sophia Bush Sends Sweet Birthday Message to 'O...   
3  Colombian singer Maluma sparks rumours of inap...   
4  Gossip Girl 10 Years Later: How Upper East Sid...   

                                            news_url        source_domain  \
0  http://toofab.com/2017/05/08/real-housewives-a...           toofab.com   
1  https://www.today.com/style/see-people-s-choic...        www.today.com   
2  https://www.etonline.com/news/220806_sophia_bu...     www.etonline.com   
3  https://www.dailymail.co.uk/news/article-33655...  www.dailymail.co.uk   
4  https://www.zerchoo.com/entertainment/gossip-g...      www.zerchoo.com   

   tweet_num  real  
0         42     1  
1          0     1  
2         63     1  
3         20     1  
4         38     1  


In [3]:
# Drop any missing titles
texts = df['title'].dropna().tolist()

# Verify
for t in texts[:5]:
    print(t)


Kandi Burruss Explodes Over Rape Accusation on 'Real Housewives of Atlanta' Reunion (Video)
People's Choice Awards 2018: The best red carpet looks
Sophia Bush Sends Sweet Birthday Message to 'One Tree Hill' Co-Star Hilarie Burton: 'Breyton 4eva'
Colombian singer Maluma sparks rumours of inappropriate relationship with AUNT
Gossip Girl 10 Years Later: How Upper East Siders Shocked the World and Changed Pop Culture Forever


In [5]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

# Initialize ChromaDB client
client = chromadb.Client(Settings(anonymized_telemetry=False))

# Create (or get) collection
collection = client.get_or_create_collection(name="news_titles")

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Embed all titles
embeddings = model.encode(texts).tolist()

# Add to ChromaDB
collection.add(
    documents=texts,
    embeddings=embeddings,
    ids=[f"doc_{i}" for i in range(len(texts))]
)


RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
cannot import name 'CompileConfig' from 'transformers.generation' (c:\Users\Marc Jay\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\__init__.py)